# Run the SONAR-netCDF4 v1 convention checker on an AZFP file

7/2/2023

In [1]:
import echopype as ep
from echopype.testing import TEST_DATA_FOLDER

from sonarnetcdf4_echopype_checker import cc

ep.__version__

'0.6.4.dev132+g439819b3.d20230609'

## Convert the target raw file to an EchoData object

In [2]:
# Rutgers glider data, with empty tilt_x and tilt_y
# ed = ep.open_raw(
#     TEST_DATA_FOLDER / 'azfp/rutgers_glider_notemperature/18011107.01A', 
#     xml_path=TEST_DATA_FOLDER / 'azfp/rutgers_glider_notemperature/18011107.XML', 
#     sonar_model='AZFP'
# )

# A more normal data, with valid tilt_x and tilt_y
ed = ep.open_raw(
    TEST_DATA_FOLDER / 'azfp/17082117.01A', 
    xml_path=TEST_DATA_FOLDER / 'azfp/17041823.XML', 
    sonar_model='AZFP'
)

## Test the `Sonar/Beam_group1` group

### Read and process the CDL

In [3]:
target_group = "Sonar/Beam_group1"

In [4]:
conv_check = cc.ConventionCDL(target_group)

### Examine the target group in the converted EchoData object

In [5]:
ed[target_group]

<xarray.Dataset>
Dimensions:                    (channel: 4, ping_time: 240, range_sample: 2650,
                                beam: 1)
Coordinates:
  * channel                    (channel) <U11 '55075-38-1' ... '55075-455-4'
  * ping_time                  (ping_time) datetime64[ns] 2017-08-21T17:05:37...
  * range_sample               (range_sample) int64 0 1 2 3 ... 2647 2648 2649
  * beam                       (beam) <U1 '1'
Data variables:
    frequency_nominal          (channel) float64 3.8e+04 1.25e+05 2e+05 4.55e+05
    backscatter_r              (channel, ping_time, range_sample, beam) int64 ...
    equivalent_beam_angle      (channel, ping_time, beam) float64 0.02213 ......
    gain_correction            (channel, ping_time, beam) int64 118 118 ... 44
    sample_interval            (channel, ping_time) float64 1.563e-05 ... 1.5...
    transmit_duration_nominal  (channel, ping_time) float64 0.001 ... 0.001
Attributes:
    beam_mode:              
    conversion_equation_t:  type_4

In [6]:
ed

<EchoData: standardized raw data from Internal Memory>
Top-level: contains metadata about the SONAR-netCDF4 file format.
├── Environment: contains information relevant to acoustic propagation through water.
├── Platform: contains information about the platform on which the sonar is installed.
├── Provenance: contains metadata about how the SONAR-netCDF4 version of the data were obtained.
├── Sonar: contains sonar system metadata and sonar beam groups.
│   └── Beam_group1: contains backscatter power (uncalibrated) and other beam or channel-specific data.
└── Vendor_specific: contains vendor-specific information about the sonar and the data.

In [7]:
ed_group_ds = ed[target_group]

In [8]:
conv_check.set_ed_group_ds(ed_group_ds)

### Set the obligation and echopype_mods filters (optional)

In [9]:
conv_check.set_obligation(None) # "M", ['M', 'MA'], None, exclude=True

In [10]:
conv_check._get_obligation_vars()

array(['angle_alongship', 'angle_athwartship', 'angle_offset_alongship',
       'angle_offset_athwartship', 'angle_sensitivity_alongship',
       'angle_sensitivity_athwartship', 'backscatter_i', 'backscatter_r',
       'beam', 'beam_direction_x', 'beam_direction_y', 'beam_direction_z',
       'beam_stabilisation', 'beam_type', 'beamwidth_twoway_alongship',
       'beamwidth_twoway_athwartship', 'channel', 'equivalent_beam_angle',
       'frequency_end', 'frequency_nominal', 'frequency_start',
       'gain_correction', 'non_quantitative_processing', 'ping_time',
       'range_sample', 'sample_interval', 'sample_time_offset', 'slope',
       'transceiver_software_version', 'transmit_bandwidth',
       'transmit_duration_nominal', 'transmit_frequency_start',
       'transmit_frequency_stop', 'transmit_power', 'transmit_type'],
      dtype=object)

`echopype_mods` handling is not fully implemented yet

In [11]:
conv_check.set_echopype_mods(None) # None

In [12]:
conv_check.echopype_mods

### Run the tests

#### Presence of expected variables

In [13]:
conv_check.test_vars_presence(test_type="expected")

****Expected variables not found in the EchoData object:
angle_alongship
angle_athwartship
angle_offset_alongship
angle_offset_athwartship
angle_sensitivity_alongship
angle_sensitivity_athwartship
backscatter_i
beam_direction_x
beam_direction_y
beam_direction_z
beam_stabilisation
beam_type
beamwidth_twoway_alongship
beamwidth_twoway_athwartship
frequency_end
frequency_start
non_quantitative_processing
sample_time_offset
slope
transceiver_software_version
transmit_bandwidth
transmit_frequency_start
transmit_frequency_stop
transmit_power
transmit_type


#### Presence of unexpected variables

In [14]:
conv_check.test_vars_presence(test_type="unexpected")

****EchoData variables not found in the CDL:


#### Variable data type

Use two types of data type comparisons: strict (specific) vs generalized.

In [15]:
conv_check.test_vars_datatype(dtype_strict=True)

****Variables with different data type from what is expected:
gain_correction: EchoData type: int64, CDL type: float32
transmit_duration_nominal: EchoData type: float64, CDL type: float32
channel: EchoData type: <U11, CDL type: object
beam: EchoData type: <U1, CDL type: object
backscatter_r: EchoData type: int64, CDL type: float32


In [16]:
conv_check.test_vars_datatype(dtype_strict=False)

****Variables with different data type from what is expected:
gain_correction: EchoData type: int64, CDL type: float32
backscatter_r: EchoData type: int64, CDL type: float32


#### Variable dimensionality

In [17]:
conv_check.test_vars_dimensionality()

****EchoData variables with dimensionality different from the CDL:


#### Attribute presence

In [18]:
conv_check.test_attrs_presence()

****Variable or global missing convention attributes:


In [19]:
conv_check.test_attrs_presence(global_attrs=True)

****Variable or global missing convention attributes:


#### Attribute values

- This will be trickier b/c of values that are not defined as static
- **NOTE:** Currently excluding `valid_range` b/c it entails comparing lists or arrays, not scalars

In [20]:
conv_check.test_attrs_value()

****Variable or global attribute with different values:
backscatter_r.units: EchoData value: count


In [21]:
conv_check.test_attrs_value(global_attrs=True)

****Variable or global attribute with different values:
global.conversion_equation_t: EchoData value: type_4
global.beam_mode: EchoData value: 


## Test the `Platform` group

### Read and process the CDL

In [22]:
target_group = "Platform"

In [23]:
conv_check = cc.ConventionCDL(target_group)

### Examine the target group in the converted EchoData object

In [24]:
ed[target_group]

<xarray.Dataset>
Dimensions:              (time1: 1, time2: 240, channel: 4)
Coordinates:
  * channel              (channel) <U11 '55075-38-1' ... '55075-455-4'
  * time1                (time1) datetime64[ns] 2017-08-21T17:05:37
  * time2                (time2) datetime64[ns] 2017-08-21T17:05:37 ... 2017-...
Data variables: (12/21)
    latitude             (time1) float64 nan
    longitude            (time1) float64 nan
    pitch                (time2) float64 nan nan nan nan nan ... nan nan nan nan
    roll                 (time2) float64 nan nan nan nan nan ... nan nan nan nan
    vertical_offset      (time2) float64 nan nan nan nan nan ... nan nan nan nan
    water_level          float64 nan
    ...                   ...
    MRU_rotation_y       float64 nan
    MRU_rotation_z       float64 nan
    position_offset_x    float64 nan
    position_offset_y    float64 nan
    position_offset_z    float64 nan
    frequency_nominal    (channel) float64 3.8e+04 1.25e+05 2e+05 4.55e+05
Attributes:
    platform_name:       
    platform_type:       
    platform_code_ICES:

In [25]:
ed

<EchoData: standardized raw data from Internal Memory>
Top-level: contains metadata about the SONAR-netCDF4 file format.
├── Environment: contains information relevant to acoustic propagation through water.
├── Platform: contains information about the platform on which the sonar is installed.
├── Provenance: contains metadata about how the SONAR-netCDF4 version of the data were obtained.
├── Sonar: contains sonar system metadata and sonar beam groups.
│   └── Beam_group1: contains backscatter power (uncalibrated) and other beam or channel-specific data.
└── Vendor_specific: contains vendor-specific information about the sonar and the data.

In [26]:
ed[target_group].vertical_offset.min().values, ed[target_group].vertical_offset.max().values

(array(nan), array(nan))

In [27]:
ed[target_group].water_level.min().values, ed[target_group].water_level.max().values

(array(nan), array(nan))

In [28]:
if ed.sonar_model == "EK80":
    ed[target_group].drop_keel_offset.min().values, ed[target_group].drop_keel_offset.max().values

In [29]:
ed_group_ds = ed[target_group]

In [30]:
conv_check.set_ed_group_ds(ed_group_ds)

### Set the obligation and echopype_mods filters (optional)

In [31]:
conv_check.set_obligation(None) # "M", ['M', 'MA'], None, exclude=True

In [32]:
conv_check._get_obligation_vars()

array(['MRU_offset_x', 'MRU_offset_y', 'MRU_offset_z', 'MRU_rotation_x',
       'MRU_rotation_y', 'MRU_rotation_z', 'channel', 'distance',
       'drop_keel_offset', 'drop_keel_offset_is_manual',
       'frequency_nominal', 'heading', 'latitude', 'longitude', 'pitch',
       'position_offset_x', 'position_offset_y', 'position_offset_z',
       'roll', 'sentence_type', 'speed_ground', 'speed_relative',
       'tilt_x', 'tilt_y', 'time1', 'time2', 'transducer_offset_x',
       'transducer_offset_y', 'transducer_offset_z', 'vertical_offset',
       'water_level', 'water_level_draft_is_manual'], dtype=object)

`echopype_mods` handling is not fully implemented yet

In [33]:
conv_check.set_echopype_mods(None) # None

In [34]:
conv_check.echopype_mods

### Run the tests

#### Presence of expected variables

In [35]:
conv_check.test_vars_presence(test_type="expected")

****Expected variables not found in the EchoData object:
distance
drop_keel_offset
drop_keel_offset_is_manual
heading
sentence_type
speed_ground
speed_relative
water_level_draft_is_manual


#### Presence of unexpected variables

In [36]:
conv_check.test_vars_presence(test_type="unexpected")

****EchoData variables not found in the CDL:


#### Variable data type

Use two types of data type comparisons: strict (specific) vs generalized.

In [37]:
conv_check.test_vars_datatype(dtype_strict=True)

****Variables with different data type from what is expected:
MRU_offset_x: EchoData type: float64, CDL type: float32
channel: EchoData type: <U11, CDL type: object
position_offset_y: EchoData type: float64, CDL type: float32
roll: EchoData type: float64, CDL type: float32
tilt_y: EchoData type: float64, CDL type: float32
position_offset_z: EchoData type: float64, CDL type: float32
tilt_x: EchoData type: float64, CDL type: float32
MRU_rotation_y: EchoData type: float64, CDL type: float32
transducer_offset_x: EchoData type: float64, CDL type: float32
MRU_offset_y: EchoData type: float64, CDL type: float32
MRU_rotation_z: EchoData type: float64, CDL type: float32
vertical_offset: EchoData type: float64, CDL type: float32
transducer_offset_y: EchoData type: float64, CDL type: float32
water_level: EchoData type: float64, CDL type: float32
MRU_rotation_x: EchoData type: float64, CDL type: float32
MRU_offset_z: EchoData type: float64, CDL type: float32
position_offset_x: EchoData type: float

In [38]:
conv_check.test_vars_datatype(dtype_strict=False)

****Variables with different data type from what is expected:


#### Variable dimensionality

In [39]:
conv_check.test_vars_dimensionality()

****EchoData variables with dimensionality different from the CDL:


#### Attribute presence

In [40]:
conv_check.test_attrs_presence()

****Variable or global missing convention attributes:
tilt_y: Missing EchoData attrs: {'comment'}
tilt_x: Missing EchoData attrs: {'comment'}


In [41]:
conv_check.test_attrs_presence(global_attrs=True)

****Variable or global missing convention attributes:


#### Attribute values

- This will be trickier b/c of values that are not defined as static
- **NOTE:** Currently excluding `valid_range` b/c it entails comparing lists or arrays, not scalars

In [42]:
conv_check.test_attrs_value()

****Variable or global attribute with different values:


In [43]:
conv_check.test_attrs_value(global_attrs=True)

****Variable or global attribute with different values:
